In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
train = pd.read_csv('train_final.csv')
test = pd.read_csv('test_final.csv')

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
train.info()

In [ ]:
test.info()

In [ ]:
print(train.shape, test.shape)

In [ ]:
train.isna().sum()

In [ ]:
test.isna().sum()

1.   В данных есть разные типы данных, нужно устранить категориальные признаки
2.   Отсутствуют пропуски
3.   Нужно восстановить колонку is_canceled
4.   Задача решается с помощью логистической регрессии
5.   Данные несбалансированные






In [14]:
train['is_canceled'].value_counts()

0    31354
1    13284
Name: is_canceled, dtype: int64